In [5]:
import os
import cv2  #compter Vision
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
 #convert pIL image to Numpy array

# Define the dataset folder path
dataset_path = 'C:/Users/sanju/OneDrive/Documents/Nitheesh/Medicinal Leaf Dataset/Segmented Medicinal Leaf Images'

# Define the fixed size for the images
img_size = (150,150,3)

# Load the dataset
data = []
labels = []
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                image_path = os.path.join(folder_path, filename)
                image = cv2.imread(image_path) #read the image file
               # image = cv2.resize(image, img_size) # resize two-integer tuple argument the width and height
                image = cv2.resize(image, (150, 150))
                # image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                # image = cv2.normalize(image.astype('float'), None, 0.0, 1.0, cv2.NORM_MINMAX)#a process that changes the range of pixel intensity values
                image = img_to_array(image)
                data.append(image)
                labels.append(folder)

# Convert the data and labels to arrays
data = np.array(data, dtype='float32')
labels = np.array(labels)

# Print the shape of the data and labels arrays
print('Data shape:', data.shape)
print('Labels shape:', labels.shape)

Data shape: (1828, 150, 150, 3)
Labels shape: (1828,)


In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
labels = onehot_encoder.fit_transform(integer_encoded)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

c:\Users\sanju\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [7]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

# Define the CNN model
model = Sequential()
base_model = VGG16(include_top=False, input_shape=(150, 150, 3))  # Use the pre-trained VGG16 model
for layer in base_model.layers:
    layer.trainable = False  # Freeze the weights of the pre-trained layers
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
# model.add(Dense(64,activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(30, activation='softmax'))  # Use softmax activation for multi-class classification

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
history = model.fit(X_train, y_train, epochs=10)


Epoch 1/10
46/46 [==============================] - 85s 2s/step - loss: 15.9424 - accuracy: 0.1744
Epoch 2/10
46/46 [==============================] - 109s 2s/step - loss: 4.7550 - accuracy: 0.3249
Epoch 3/10
46/46 [==============================] - 80s 2s/step - loss: 3.1254 - accuracy: 0.4083
Epoch 4/10
46/46 [==============================] - 3002s 67s/step - loss: 2.3410 - accuracy: 0.4952
Epoch 5/10
46/46 [==============================] - 65s 1s/step - loss: 2.1926 - accuracy: 0.5335
Epoch 6/10
46/46 [==============================] - 79s 2s/step - loss: 1.7676 - accuracy: 0.6074
Epoch 7/10
46/46 [==============================] - 95s 2s/step - loss: 1.6135 - accuracy: 0.6245
Epoch 8/10
46/46 [==============================] - 143s 3s/step - loss: 1.3007 - accuracy: 0.7107
Epoch 9/10
46/46 [==============================] - 147s 3s/step - loss: 1.1355 - accuracy: 0.7230
Epoch 10/10
46/46 [==============================] - 146s 3s/step - loss: 1.0154 - accuracy: 0.7497


In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Evaluate the model on the testing set
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Calculate the accuracy, precision, recall, and F1-score
accuracy = accuracy_score(y_test_classes, y_pred_classes)
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
confusion = confusion_matrix(y_test_classes, y_pred_classes)
print("Accuracy is :" ,accuracy)
print("Precision is :" ,precision)
print("Recall is :",recall)
print("F1 Score" ,f1)

12/12 [==============================] - 26s 2s/step
Accuracy is : 0.8961748633879781
Precision is : 0.917450809392447
Recall is : 0.8961748633879781
F1 Score 0.8897807101427696
